In [1]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math, statsmodels.api as sm, patsy as ps
from tqdm import tqdm
from time import process_time
from datetime import datetime
from datetime import date
from openpyxl import load_workbook
from patsy import dmatrices

############### Set working directory to parent directory
if os.getcwd() != 'F:\\github\\narrative_conservatism\\code':
    os.chdir('F:\\github\\narrative_conservatism\\code')

############### Set pandas column printing constraint
pd.set_option('display.max_columns', None)

In [2]:
#############################################################################################
####### Merge IBES annual data (oen-year forecast) with CRSP_COMP_EDGAR quarterly data ######
#############################################################################################

########### Read IBES raw data file
ibes_cols = ['cusip8', 'cname', 'fpi', 'value', 'fpedats', 'anndats', 'actual', 'anndats_act']
ibes = pd.read_csv('..\\filings\\ibes.csv', usecols = ibes_cols)
print('number of cusip-fpedats-analyst: ' + str(ibes.shape[0]))

### Reorder and rename IBES columns
ibes = ibes[['cusip8', 'cname', 'fpedats', 'value', 'actual', 'anndats_act', 'anndats', 'fpi']]
ibes = ibes.rename(columns={'cusip8': 'cusip'})

### correct fpedats format
ibes['fpedats'] = ibes['fpedats'].str.replace('jan','01')
ibes['fpedats'] = ibes['fpedats'].str.replace('feb','02')
ibes['fpedats'] = ibes['fpedats'].str.replace('mar','03')
ibes['fpedats'] = ibes['fpedats'].str.replace('apr','04')
ibes['fpedats'] = ibes['fpedats'].str.replace('may','05')
ibes['fpedats'] = ibes['fpedats'].str.replace('jun','06')
ibes['fpedats'] = ibes['fpedats'].str.replace('jul','07')
ibes['fpedats'] = ibes['fpedats'].str.replace('aug','08')
ibes['fpedats'] = ibes['fpedats'].str.replace('sep','09')
ibes['fpedats'] = ibes['fpedats'].str.replace('oct','10')
ibes['fpedats'] = ibes['fpedats'].str.replace('nov','11')
ibes['fpedats'] = ibes['fpedats'].str.replace('dec','12')
ibes['fpedats'] = pd.to_datetime(ibes['fpedats'], format='%d%m%Y')

print(ibes['fpedats'].describe())

### Delete missing cusip8 and actual
del_cusip = ibes[ibes['cusip'].isnull()].shape[0]
ibes = ibes[ibes['cusip'].isnull() == False]
print('number of obs. that contain missing cusip: ' + str(del_cusip))

del_actual = ibes[ibes['actual'].isnull()].shape[0]
ibes = ibes[ibes['actual'].isnull() == False]
print('number of obs. that contain missing actual: ' + str(del_actual))

### aggregate by cusip-fpedats and get actual, median, afe and consensus from ibes raw data
ibes_css = ibes.groupby(['cusip', 'fpedats'])['actual'].median().to_frame()
ibes_css['median'] = ibes.groupby(['cusip', 'fpedats'])['value'].median().to_frame()
ibes_css['afe'] = ibes_css['actual'] - ibes_css['median']
ibes_css['consensus'] = ibes.groupby(['cusip', 'fpedats'])['value'].mean().to_frame()
ibes_css['leap_consensus'] = ibes_css.groupby(['cusip'])['consensus'].shift(-1)

### make a consensus forecast dataset by cusip-fpedats
ibes = ibes.loc[ibes.duplicated(subset=['cusip', 'fpedats']) == False, ['cusip','fpedats']]
ibes = ibes.join(ibes_css, on=['cusip', 'fpedats'])

### create merge date_key - keep %y only
ibes['date_key'] = ibes['fpedats'].astype(str).str[:-6].astype(int)

print('number of cusip-fpedats, dropping missing cusip and actual: ' + str(ibes.shape[0]))

number of cusip-fpedats-analyst: 9812071
count                 9812071
unique                    446
top       2015-12-31 00:00:00
freq                   426980
first     1981-12-31 00:00:00
last      2019-07-31 00:00:00
Name: fpedats, dtype: object
number of obs. that contain missing cusip: 60781
number of obs. that contain missing actual: 1305946
number of cusip-fpedats, dropping missing cusip and actual: 155539


In [3]:
##################### Read EDGAR_CRSP_COMP and creat merge date_key
crsp_comp_edgar = pd.read_csv('..\\filings\\crsp_comp_edgar_10-Q.csv')
crsp_comp_edgar['date_key'] = crsp_comp_edgar['fyearq'].astype(int)

##################### Inner merge EDGAR_CRSP_COMP and IBES, key not unique in both datasets
crsp_comp_edgar_ibes = pd.merge(crsp_comp_edgar, ibes, on = ['cusip', 'date_key'], how='inner', validate = 'm:m')
print('number of cusip-date after merging: ' + str(crsp_comp_edgar_ibes.shape[0]))

crsp_comp_edgar_ibes.sort_values(by = ['cusip', 'cquarter'])

number of cusip-date after merging: 110096


,cusip,cik,ticker,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,aqcy,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,sstky,xrdq,HHI,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,lag1_CFO,lag2_CFO,lag3_CFO,LAG1_RET,LAG2_RET,LAG3_RET,TACC,LAG_TA_REV,DSAR,PPE,SIC2,SG,LAG_SG,SKEW_RET,TURNOVER,nw,nvocab,n_neg,n_pos,n_neg_gi,n_pos_gi,n_neg_he,n_pos_he,n_uctt,n_lit,n_cstr,n_modal_week,n_modal_moderate,n_modal_weak,n_negation,READ,tone,tone_gi,tone_he,NW,TONE,TONE_GI,TONE_HE,TLAG,NEG,SIZE,MTB,LEV,AGE,date_key,fpedats,actual,median,afe,consensus,leap_consensus
107232,00030710,1606180,AACH,2015-03-31,0001564590-15-003247,"AAC Holdings, Inc.",178698.0,8060,2015-05-05,2015-03-31,2015-03-31,201503,2015.0,1.0,NV,NV,37027,A,182,99.585,14.102,59.186,1.340,2.038,26.089,17.742,42.823,0.749,NaN,0.000,0.0,0.002239,188.983,145.952,101.145,97.474,21.781,21.374,4.440,4.357,71.628,24.284,30.580,30.9200,2.038,3.354,0.066935,0.234733,0.013963,0.0,-0.009017,NaN,-0.072387,0.021950,0.062143,NaN,0.017252,NaN,NaN,12.603,0.699826,-0.022871,0.484228,80,0.029934,0.003885,0.706554,93559.0,16287,1709,163,47,161,800,50,154,193,82,102,24,26,32,1,24.89,-0.007184,0.039172,0.006324,9.698184,-7.183643,39.172346,6.324062,35,0,6.493578,6.780106,0.196236,5.209486,2015,2015-12-31,0.9700,0.7800,0.1900,0.753333,NaN
107233,00030710,1606180,AACH,2015-06-30,0001564590-15-005942,"AAC Holdings, Inc.",178698.0,8060,2015-08-03,2015-06-30,2015-06-30,201506,2015.0,2.0,NV,NV,37027,A,273,97.666,14.251,45.021,1.676,5.555,28.972,30.118,53.784,0.303,NaN,0.000,0.0,0.002239,210.265,188.983,109.824,101.145,21.816,21.781,5.227,4.440,70.641,71.628,43.560,30.5800,5.555,2.038,0.375746,0.011999,0.029394,0.0,0.018610,NaN,0.048258,-0.072387,0.021950,0.062143,0.066935,0.017252,NaN,-3.565,0.772302,0.007080,0.491055,80,0.052129,0.029934,0.067869,126640.0,21532,2044,305,85,333,982,93,276,301,182,138,32,33,91,13,27.31,-0.010821,0.029537,0.007895,9.977342,-10.821103,29.537433,7.895226,34,0,6.501384,6.585229,0.402512,5.613128,2015,2015-12-31,0.9700,0.7800,0.1900,0.753333,NaN
107234,00030710,1606180,AACH,2015-09-30,0001564590-15-010373,"AAC Holdings, Inc.",178698.0,8060,2015-11-10,2015-09-30,2015-09-30,201509,2015.0,3.0,NV,NV,37027,A,365,76.724,84.471,10.764,1.921,2.452,112.594,32.120,57.372,1.942,NaN,0.000,0.0,0.002239,290.911,210.265,139.720,109.824,22.407,21.816,4.856,5.227,116.710,70.641,22.250,43.5600,2.452,5.555,-0.482160,0.121349,0.011661,0.0,-0.014758,0.009461,-0.003657,0.048258,-0.072387,0.021950,0.375746,0.066935,0.017252,3.221,0.898785,-0.034033,0.518812,80,0.012334,0.052129,-0.256675,399243.0,21050,1947,232,68,247,942,72,246,214,140,108,27,32,39,7,29.19,-0.008124,0.032684,0.007933,9.954703,-8.123515,32.684086,7.933492,41,1,6.856783,8.652981,0.360821,5.902633,2015,2015-12-31,0.9700,0.7800,0.1900,0.753333,NaN
9664,00036020,824142,AAON,1996-09-30,0000106455-96-000015,AAON INC,21542.0,3585,1996-11-01,1996-09-30,1996-09-30,199609,1996.0,3.0,NV,NV,74107,A,2100,26.423,0.000,0.025,0.621,0.522,NaN,13.827,17.173,0.000,1.796,0.010,NaN,0.002271,36.501,33.936,14.972,14.450,6.122,6.122,0.172,0.714,7.702,10.217,5.625,5.2500,0.522,0.475,0.038653,0.046985,0.015382,0.0,0.001385,0.005044,0.100660,-0.051222,-0.094561,0.045195,0.072283,-0.224736,-0.137973,-2.894,0.888820,-0.029143,0.501680,35,0.050355,0.055899,0.549105,5523.0,1276,450,5,5,8,50,5,16,9,9,6,2,4,2,0,112.69,0.000000,0.032915,0.008621,7.152269,0.000000,32.915361,8.620690,32,0,3.470117,2.224256,0.322106,7.650169,1996,1996-12-31,0.0307,0.0702,-0.0395,0.071073,0.045367
25415,00036020,824142,AAON,1999-03-31,0001026608-99-000042,AAON INC,21542.0,3585,1999-05-14,1999-03-31,1999-03-31,199903,1999.0,1.0,NV,NV,74107,A,3012,31.559,0.000,0.369,0.732,1.764,NaN,15.500,30.036,NaN,4.266,0.048,NaN,0.002271,50.827,50.506,26.223,24.411,6.227,6.219,0.757,0.757,9.104,10.980,11.000,9.3125,1.764,1.456,0.167819,0.136016,0.034

In [4]:
########################################################################################
############ Merge CRSP_COMP_EDGAR_IBES data with SEG data #############################
########################################################################################

########### Read SEG raw data file
seg_cols = ['gvkey', 'stype', 'sid', 'datadate', 'snms', 'cusip', 'cik']
seg = pd.read_csv('..\\filings\\compustat_seg.csv', usecols = seg_cols)
print('number of cusip-fpedats-analyst: ' + str(seg.shape[0]))

### Reorder and rename IBES columns
seg = seg[['gvkey', 'cik', 'cusip', 'datadate', 'stype', 'sid', 'snms']]
seg = seg.rename(columns={'datadate': 'date_comp'})

### parse date_comp format
seg['date_comp'] = pd.to_datetime(seg['date_comp'], format='%Y%m%d')

### aggregate by gvkey-date_comp-stype and get count of sid
seg_count = seg.groupby(['gvkey', 'date_comp', 'stype'])['sid'].count().to_frame()

### make a segment dataset by gvkey-date_comp-stype
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp', 'stype']) == False, ['gvkey', 'date_comp', 'stype']]
seg = seg.join(seg_count, on=['gvkey', 'date_comp', 'stype'])
seg_bus = seg[seg['stype']=='BUSSEG']
seg_bus = seg_bus.drop(columns=['stype'])
seg_bus = seg_bus.rename(columns={'sid': 'nseg_bus'})
seg_geo = seg[seg['stype']=='GEOSEG']
seg_geo = seg_geo.drop(columns=['stype'])
seg_geo = seg_geo.rename(columns={'sid': 'nseg_geo'})
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp']) == False, ['gvkey', 'date_comp']]
seg = pd.merge(seg, seg_bus, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = pd.merge(seg, seg_geo, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = seg.sort_values(by=['gvkey', 'date_comp'])
seg.loc[seg['nseg_geo'].isnull(), 'nseg_geo'] = 1
seg.loc[seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
seg['nseg_bus'] = seg['nseg_bus'].astype(int)
seg['nseg_geo'] = seg['nseg_geo'].astype(int)

### create merge date_key - %y only
seg['date_key'] = seg['date_comp'].astype(str).str[:-6].astype(int)
seg = seg.drop(columns=['date_comp'])

print('number of gvkey-datadate in SEGMENT data: ' + str(seg.shape[0]))

number of cusip-fpedats-analyst: 452653
number of gvkey-datadate in SEGMENT data: 50876


In [5]:
##################### left merge EDGAR_CRSP_COMP_IBES and SEG, key not unique in both datasets
crsp_comp_edgar_ibes_seg = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='left', validate = 'm:m')
# crsp_comp_edgar_ibes_seg_inner = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='inner', validate = 'm:m')
# print('Number of gvkey-quarter obs. after merging, inner: '+ str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

##################### Fill missing segments in crsp_comp_edgar_ibes_seg_left with 1
crsp_comp_edgar_ibes_seg.loc[crsp_comp_edgar_ibes_seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
crsp_comp_edgar_ibes_seg.loc[crsp_comp_edgar_ibes_seg['nseg_geo'].isnull(), 'nseg_geo'] = 1

####### Drop duplicated gykey-cquarter
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg.duplicated(subset=['gvkey', 'cquarter']) == False]
print('Number of gvkey-quarter obs. after merging, left: '+ str(crsp_comp_edgar_ibes_seg.shape[0]))

Number of gvkey-quarter obs. after merging, left: 110062


In [6]:
########################################################################################
############################### Variable Creation ######################################
########################################################################################

################### CRSP_COMP_EDGAR_IBES_SEG_LEFT ######################################

######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
crsp_comp_edgar_ibes_seg['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg['age'])

######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
crsp_comp_edgar_ibes_seg['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg['nseg_bus'])
crsp_comp_edgar_ibes_seg['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg['nseg_geo'])

######## AFE, analyst forecast error, \
######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
######## deflated by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg['AFE'] = crsp_comp_edgar_ibes_seg['afe']/crsp_comp_edgar_ibes_seg['prccq']

######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg['AF'] = crsp_comp_edgar_ibes_seg['leap_consensus']/crsp_comp_edgar_ibes_seg['prccq']

In [7]:
########################################################################################
########################## Variable Screening ##########################################
########################################################################################

########## Drop files (firm-quarter) that have positive/negative infinity DEARN
del_DEARN = crsp_comp_edgar_ibes_seg[(crsp_comp_edgar_ibes_seg['DEARN']==np.inf) | (crsp_comp_edgar_ibes_seg['DEARN']==-np.inf)].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[(crsp_comp_edgar_ibes_seg['DEARN']!=np.inf) & (crsp_comp_edgar_ibes_seg['DEARN']!=-np.inf)]
print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

### Drop missing AF
del_AF = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AF'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AF'].isnull() == False]
print('number of obs. that contain missing AF: ' + str(del_AF))

### Drop missing AFE
del_AFE = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AFE'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AFE'].isnull() == False]
print('number of obs. that contain missing AFE: ' + str(del_AFE))

### Drop missing EARN
del_EARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['EARN'].isnull() == False]
print('number of obs. that contain missing EARN: ' + str(del_EARN))

### Drop missing STD_EARN
del_STD_EARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['STD_EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['STD_EARN'].isnull() == False]
print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

### Drop missing EARN
del_DEARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DEARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DEARN'].isnull() == False]
print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

############## Winsorize ##############################
###### Define a function that winsorize a variable at 1% and 99% 
def winsorize (df, colnames):
    for col in colnames:
        varq01 = df[col].quantile(.01)
        varq99 = df[col].quantile(.99)
        df[col] = df[col].clip(varq01, varq99)
    return df

############## Winsorize TABLE 1 (line 1) and TABLE 4 (line 2) variables
crsp_comp_edgar_ibes_seg = winsorize(crsp_comp_edgar_ibes_seg, \
                                          ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', 'SIZE', 'MTB', 'LEV', \
                                          'CFO', 'lag1_CFO', 'lag2_CFO', 'lag3_CFO', 'READ'])

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

number of files that have positive/negative infinity DEARN: 0
number of obs. that contain missing AF: 15230
number of obs. that contain missing AFE: 0
number of obs. that contain missing EARN: 64
number of obs. that contain missing STD_EARN: 3162
number of obs. that contain missing DEARN: 0
Number of firm-quarters after variable screening: 91606


In [8]:
# ######## Create ABTONE: residual from TONE regression 
# y, X = ps.dmatrices('TONE ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
#                     data = crsp_comp_edgar_ibes_seg, return_type = 'dataframe')
# model = sm.OLS(y, X)
# res = model.fit()
# # res.summary()
# crsp_comp_edgar_ibes_seg['ABTONE'] = res.resid

# ######## Create abtone for REPLICATION: residual from tone regression 
# y, X = ps.dmatrices('tone ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
#                     data = crsp_comp_edgar_ibes_seg, return_type = 'dataframe')
# model = sm.OLS(y, X)
# res = model.fit()
# # res.summary()
# crsp_comp_edgar_ibes_seg['abtone'] = res.resid

############## Save merged CRSP_COMP_EDGAR_IBES_SEG to csv file
crsp_comp_edgar_ibes_seg.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

crsp_comp_edgar_ibes_seg

,cusip,cik,ticker,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,aqcy,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,sstky,xrdq,HHI,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,lag1_CFO,lag2_CFO,lag3_CFO,LAG1_RET,LAG2_RET,LAG3_RET,TACC,LAG_TA_REV,DSAR,PPE,SIC2,SG,LAG_SG,SKEW_RET,TURNOVER,nw,nvocab,n_neg,n_pos,n_neg_gi,n_pos_gi,n_neg_he,n_pos_he,n_uctt,n_lit,n_cstr,n_modal_week,n_modal_moderate,n_modal_weak,n_negation,READ,tone,tone_gi,tone_he,NW,TONE,TONE_GI,TONE_HE,TLAG,NEG,SIZE,MTB,LEV,AGE,date_key,fpedats,actual,median,afe,consensus,leap_consensus,nseg_bus,nseg_geo,BUSSEG,GEOSEG,AFE,AF
0,54626810,60512,LLX.,1993-06-30,0000060512-94-000005,LOUISIANA LAND & EXPLORATION CO,6819.0,1311,1993-08-13,1993-06-30,1993-06-30,199306,1993.0,2.0,MD,MD,70112,I,11324,193.800,0.0,64.300,27.300,5.600,NaN,208.200,189.300,136.100,23.900,0.000,NaN,0.001371,1278.000,1203.400,424.200,421.300,28.729,28.647,86.500,0.000,356.300,379.500,42.500,45.250,5.600,2.700,-0.059108,0.054464,0.004653,0.0,0.002410,0.002744,0.031660,-0.127616,0.063406,0.024354,0.261217,-0.153146,0.084896,-32.500,1.004737,-0.000249,1.996925,13,0.004304,-0.010886,0.342200,32253.0,3187,743,66,6,60,112,24,34,23,43,6,9,6,6,0,262.519,-0.018826,0.016316,0.003138,8.067149,-18.826483,16.316285,3.137747,44,1,7.167251,3.076850,0.315356,9.334768,1993,1993-12-31,0.52,1.00,-0.48,0.990359,0.714757,1.0,1.0,0.693147,0.693147,-0.011294,0.016818
1,54626810,60512,LLX.,1993-09-30,0000060512-94-000007,LOUISIANA LAND & EXPLORATION CO,6819.0,1311,1993-11-10,1993-09-30,1993-09-30,199309,1993.0,3.0,MD,MD,70112,I,11416,180.200,359.1,9.400,28.200,-1.800,NaN,195.100,187.900,124.800,23.700,0.000,NaN,0.001371,1662.300,1278.000,417.100,424.200,33.137,28.729,0.000,86.500,747.700,356.300,44.625,42.500,-1.800,5.600,0.025969,0.094378,-0.001408,1.0,-0.005790,0.004136,0.037089,0.031660,-0.127616,0.063406,-0.059108,0.261217,-0.153146,-49.200,0.941628,-0.031768,2.206729,13,-0.000842,0.004304,0.699556,39053.0,3965,789,71,13,68,143,25,54,23,56,3,16,8,4,0,262.519,-0.014628,0.018916,0.007314,8.285513,-14.627995,18.915511,7.313997,41,0,7.107411,2.878318,0.346479,9.342859,1993,1993-12-31,0.52,1.00,-0.48,0.990359,0.714757,1.0,1.0,0.693147,0.693147,-0.010756,0.016017
2,08750910,11860,3BHMSQ,1993-09-30,0000011860-94-000005,BETHLEHEM STEEL CORP /DE/,2189.0,3312,1993-11-12,1993-09-30,1993-09-30,199309,1993.0,3.0,DE,DE,18016-7699,I,24715,1074.900,0.0,232.500,69.100,30.700,NaN,908.400,1055.300,0.000,38.400,248.500,NaN,0.001801,5168.098,5463.199,580.800,958.900,91.307,91.025,100.300,88.400,728.500,650.200,14.375,18.750,30.700,-13.600,-0.270146,0.119124,0.005619,0.0,0.008109,0.007031,0.007871,0.014283,-0.036208,0.022668,0.058839,0.073953,0.276972,-12.300,1.004723,-0.008823,1.315529,33,-0.012016,0.017755,0.227962,488805.0,4032,940,81,38,90,188,25,68,23,62,32,19,12,5,0,94.770,-0.010665,0.024306,0.010665,8.302266,-10.664683,24.305556,10.664683,43,1,7.442328,1.779871,0.135196,10.115206,1993,1993-12-31,0.41,0.30,0.11,0.610301,1.489195,1.0,1.0,0.693147,0.693147,0.007652,0.103596
3,91335310,101929,UVX,1993-11-30,0000101929-94-000002,UNIVAR CORP,11003.0,5160,1994-01-13,1993-11-30,1993-11-30,199311,1993.0,3.0,DE,DE,98033,I,9041,372.727,0.0,18.416,6.890,0.458,NaN,281.853,430.299,NaN,60.117,0.290,0.00,0.010732,654.434,660.336,160.280,161.722,19.643,19.635,42.298,47.584,150.563,152.012,11.000,13.250,0.458,1.957,-0.170798,0.066070,0.000694,0.0,-0.002270,0.002216,0.037449,-0.002659,-0.024944,0.018006,0.138301,0.037491,-0.099459,-24.271,1.084569,-0.049554,NaN,51,-0.066957,-0.020948,0.672838,7200.0,2795,638,20,8,19,100,11,16,12,62,13,2,5,3,0,82.140,-0.004293,0.028980,0.001789,7.935945,-4.293381,28.980322,1.788909,44,1,5.561311,1.608710,0.302264,9.109636,1993,1993-02-28,0.27,0.58,-0.31,0.636296,0.586216,1.0,1.0,0.693147,0.693147,-0.028182,0.053292
4,91345610,102037,UVV,1993-12-31,0000950118-94-0000

In [9]:
# ####################################################################################################################
# ############# Create COMP_DA: if not replicating TABLE 4, DO NOT run because cause large sample reduction ##########
# ####################################################################################################################

# ### Drop missing or infinite LAG_TA_REV, DSAR and PPE
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['LAG_TA_REV'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['LAG_TA_REV'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['LAG_TA_REV'] != -np.inf)]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['DSAR'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['DSAR'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['DSAR'] != -np.inf)]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['PPE'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['PPE'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['PPE'] != -np.inf)]
# print('number of obs. after deleting missing or infinite LAG_TA_REV, DSAR and PPE: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

# ############## Winsorize DA variables
# crsp_comp_edgar_ibes_seg = winsorize(crsp_comp_edgar_ibes_seg, ['TACC', 'LAG_TA_REV', 'DSAR', 'PPE'])

# ######## Create DA: residual from TACC regression by each two-digit SIC-year
# def TACC_reg(data):
#     try:
#         y, X = ps.dmatrices('TACC ~ 1 + LAG_TA_REV + DSAR + PPE', data = data, return_type = 'dataframe')
#         model = sm.OLS(y, X)
#         res = model.fit()
#         data['DA'] = res.resid
#         return data
#     except:
#         pass

# crsp_comp_edgar_ibes_seg1 = crsp_comp_edgar_ibes_seg.groupby(['SIC2','fyearq']).apply(TACC_reg)
# # comp_DA['DA'].describe()

# ####### Join DA to COMP
# crsp_comp_edgar_ibes_seg = pd.merge(crsp_comp_edgar_ibes_seg, crsp_comp_edgar_ibes_seg1[['cik', 'rp', 'DA']], on = ['cik', 'rp'], how='left', validate = '1:1')

# crsp_comp_edgar_ibes_seg

# ############## Dropping missing DA to replicate Huang et al. 2014 TABLE 4, \
# del_DA = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DA'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DA'].isnull() == False]
# print('number of obs. that contain missing DA: ' + str(del_DA))

# ############## Inspect sample size after variable screening
# print('Number of firm-quarters after dropping missing DA: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

# ############## Save merged ID_CRSP_COMP_TEXT to csv file
# crsp_comp_edgar_ibes_seg.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_DA_10-Q.csv', index = 0)

In [10]:
########################################################################################
########### Table 2 - Panel A: Summary Statistics and Correlation Matrix ###############
########################################################################################

############# Table 2 Panel A: Summary statistics for selected variables
######### Variable groups:
# 1st line: textual variables, generally consistent with LM's summary statistics
# 2nd line: fundamental variables (main)
# 3rd line: abtone
selected_vars = crsp_comp_edgar_ibes_seg[['NW','nw', 'TONE','TONE_GI', 'TONE_HE', 'TLAG', 'READ', \
                                          'RET', 'NEG', 'SIZE', 'MTB', 'LEV', \
                                          'AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'STD_EARN', 'STD_RET', 'LOSS'\
                                        # 'DEARN', 'LOSS', ABTONE', 'DA' 
                                        ]]

T2PA = selected_vars.describe().transpose() 

############ count number of unique firms
print('Number of unique firms in final 10-Q sample: ' + str(crsp_comp_edgar_ibes_seg['cik'].unique().shape[0]))

############# Summary statistics for all raw and processed variables
full_summary = crsp_comp_edgar_ibes_seg.describe().transpose()

############# Save T3PA
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T2PA.to_excel(writer, sheet_name='T2PA_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T2PA.to_excel(table_path, sheet_name='T2PA_raw', float_format="%.4f")

T2PA

Number of unique firms in final 10-Q sample: 5250


,count,mean,std,min,25%,50%,75%,max
NW,91606.0,9.020033,0.756982,7.120444,8.505576,9.085910,9.546670,13.544145
nw,91606.0,10937.344541,10203.939643,1236.000000,4941.250000,8829.000000,13997.000000,762337.000000
TONE,91606.0,-8.920548,7.235669,-63.578565,-13.127278,-7.874914,-3.865979,24.214907
TONE_GI,91606.0,24.258823,7.134229,-21.510297,19.549418,24.046339,28.728095,93.200190
TONE_HE,91606.0,6.607627,5.234760,-26.068577,3.381871,6.085193,9.309634,55.884287
TLAG,91606.0,39.020348,6.249381,0.000000,36.000000,40.000000,44.000000,52.000000
READ,91606.0,38.161176,42.160152,14.580000,17.840000,20.210000,39.660000,262.519000
RET,91606.0,0.018278,0.253080,-1.578704,-0.112614,0.007312,0.129934,4.849226
NEG,91606.0,0.482905,0.499710,0.000000,0.000000,0.000000,1.000000,1.000000
SIZE,91606.0,6.451703,1.771743,2.897718,5.175087,6.317271,7.562846,11.315066


In [11]:
# full_summary

In [12]:
############# Table 3 Panel B: Correlation matrix for selected variables
######### pearson correlation
T2PC_pearson = selected_vars.loc[:, (selected_vars.columns != 'nw') & \
                                 (selected_vars.columns != 'BUSSEG') & \
                                 (selected_vars.columns != 'GEOSEG') & \
                                 (selected_vars.columns != 'LOSS')].corr(method='pearson')

# T2PC_pearson

In [13]:
######### spearman correlation
T2PC_spearman = selected_vars.loc[:, (selected_vars.columns != 'nw') & \
                                 (selected_vars.columns != 'BUSSEG') & \
                                 (selected_vars.columns != 'GEOSEG') & \
                                 (selected_vars.columns != 'LOSS')].corr(method='spearman')

# T2PC_spearman

In [14]:
######### Combine two correlation matrices. right-up matrix: pearson; left-down matrix: spearman 
for row in list(range(0, len(T2PC_spearman.index))):
    T2PC_spearman.iloc[row, row+1:] = T2PC_pearson.iloc[row, row+1:]
    
##### Save T2PC
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T2PC_spearman.to_excel(writer, sheet_name='T2PC_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T2PC_spearman.to_excel(table_path, sheet_name='T2PC_raw', float_format="%.4f")

T2PC_spearman

,NW,TONE,TONE_GI,TONE_HE,TLAG,READ,RET,NEG,SIZE,MTB,LEV,AF,AFE,AGE,EARN,STD_EARN,STD_RET
NW,1.000000,-0.455695,0.004574,-0.248386,-0.192175,-0.082586,-0.006470,0.002258,0.254655,0.058080,0.035874,-0.067767,0.010801,-0.040219,-0.116295,0.091267,-0.030267
TONE,-0.481465,1.000000,0.417468,0.377476,0.015539,0.086126,0.020358,-0.020809,-0.061748,-0.012885,0.071853,0.071547,0.101820,0.059052,0.157343,-0.148010,-0.088735
TONE_GI,-0.021919,0.393862,1.000000,0.158465,0.012747,0.096438,0.017884,-0.014256,-0.010546,0.076404,0.145672,-0.019369,0.073964,-0.053779,0.011707,-0.015178,-0.041669
TONE_HE,-0.229834,0.390286,0.168281,1.000000,-0.007025,0.050930,0.031697,-0.048573,0.142957,0.106900,0.042930,0.063681,0.186521,0.022252,0.231949,-0.155414,-0.095522
TLAG,-0.263261,0.021265,0.001837,0.000757,1.000000,0.047966,-0.022210,0.034012,-0.330894,-0.022953,0.009258,-0.092250,-0.127343,-0.228377,-0.136763,0.121033,0.189194
READ,-0.251934,0.169253,0.146803,0.068430,0.124631,1.000000,-0.016435,0.015691,-0.014113,-0.036524,0.062913,0.045047,0.002047,0.088267,0.059172,-0.047389,-0.050854
RET,-0.006924,0.028307,0.018351,0.054292,-0.032299,-0.028800,1.000000,-0.684122,-0.063789,-0.028910,0.001817,-0.018170,0.154626,0.001701,0.063448,0.010800,0.266062
NEG,0.003387,-0.023511,-0.015089,-0.051615,0.032698,0.028255,-0.865519,1.000000,-0.000109,0.014347,-0.001902,0.015166,-0.124061,-0.017914,-0.071294,0.015659,-0.117616
SIZE,0.263966,-0.046530,-0.005283,0.149122,-0.332569,-0.077924,-0.024162,-0.000999,1.000000,0.247198,0.101096,0.078913,0.266816,0.345020,0.259155,-0.197720,-0.309537
MTB,0.046297,0.039937,0.090729,0.207440,-0.042138,-0.025543,-0.054696,0.032959,0.382006,1.000000,0.033285,-0.166468,0.128282,-0.093970,-0.040000,0.163157,0.036732
